In [1]:
# Setup the Jupyter version of Dash

# Author: Chris Trimmer
# Course: CS340 Client/Server Development
# Assignment: Week 7 Project
# Date: 10/11/2023


# Configure the necessary Python module imports for dashboard components
from dash import Dash, callback
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc

# imports for maps and charts
import dash_leaflet as dl
import plotly.express as px

# Configure OS routines
import os

# use for images
import base64

# use for regex
import re

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# Use autoreload so you don't have to continually reload the lib if there was a change
%load_ext autoreload
%autoreload 2
from CRUDLibWeek7 import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "aac1234"
host = 'nv-desktop-services.apporto.com'
port = 31321
crudModuleName = 'CRUDLibWeek7'

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.readAll({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
#app = JupyterDash(__name__)
app = Dash(__name__)


#FIX ME Add in Grazioso Salvare’s logo
image_filename2 = "GraziosoLogo.png"  # replace with your own image
encoded_image_logo = base64.b64encode(open(image_filename2, "rb").read())


#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
# Instantiate image object to represent unique identifier
image_filename = 'shepherd.jpg' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


# Main screen layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),

    # Display unique logo, name/project, and Grazioso logo
    # Note the Grazioso logo is clickable to https://www.snhu.edu
    html.Center(
        [
            # display unique logo
            html.A(
                children=[
                    html.Img(
                        alt="Chris Trimmer Identifier",
                        src='data:image/jpg;base64,{}'.format(encoded_image.decode()),
                        style={'height': '80px', 'width': '80px',
                               'textAlign': 'center', "display": "inline-block"}
                    )
                ]
            ),

            # display name and project title
            html.H2(children='Chris Trimmer - Grazioso Dashboard', style={"display":"inline-block", 
                                                          'margin-left':'70px',
                                                         'verticalAlign':'top'}
            ),

            # display Grazioso logo and set reference to snhu website
            html.A(
              href = "https://www.snhu.edu", target="_blank",
                children=[
                    html.Img(
                        alt="Link to Grazioso",
                        src='data:image/jpg;base64,{}'.format(encoded_image_logo.decode()),
                        style={'height':'90px', 'width':'90px', 'textAlign':'center',
                               'margin-left':'70px', "display":"inline-block"}
                    )
                ]
            ),
         ]
    ),
    
    #FIXME Add in code for the interactive filtering options. 
    # For example, Radio buttons, drop down, checkboxes, etc.
    # display horizontal line break and then the radio button panel (displayed horizontally)
    # currently there are four options: Water, Mountain/Wilderness, Disaster/Tracking, Reset Pages
    # the default is the Reset Pages option
    html.Hr(),
    html.Center(html.Div(
        dcc.RadioItems (
            id='filter-type',
            options=[
                {'label':html.Div(['Water Rescue'], style={'color':'Blue', 
                                                           'minWidth':'200px','width':'200px',
                                                           'maxWidth':'200px'}),
                 'value':'water',
                },
                {'label':html.Div(['Mountain/Wilderness'], style={'color':'Green',
                                                                  'minWidth':'200px','width':'200px',
                                                                  'maxWidth':'200px'}),
                 'value':'wild',
                },
                {'label':html.Div(['Disaster/Tracking'], style={'color':'Red',
                                                                'minWidth':'200px','width':'200px',
                                                                'maxWidth':'200px'}),
                 'value':'disaster',
                },
                {'label':html.Div(['All Dogs'], style={'color':'peru',
                                                    'minWidth':'200px', 'width':'200px',
                                                    'maxWidth':'200px'}),
                 'value':'dogs',
                },
                {'label':html.Div(['Reset Pages'], style={'color':'Black',
                                                    'minWidth':'200px', 'width':'200px',
                                                    'maxWidth':'200px'}),
                 'value':'reset',
                },
            ], value = 'reset', inline = True
        )
    )),

    # Format and Display table data with appropriate options and settings
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        style_header={
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
            'textAlign':'left',
        },
        
        style_data= {
            'textAlign':'left',
        },
        
        style_cell={
            'overflow':'hidden',
            'textOverflow':'ellipsis',
        },
        
        style_table={
            'overflowX':'auto'
        },
        
        # set the data
        data=df.to_dict('records'),

        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode='multi',
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=5,
        style_as_list_view=True,
    ),
    html.Br(),
    
    # Display pie chart and map below the datatable
    html.Hr(),
    html.Div(
        className='row',
        style={'display':'flex'},
            children=[
                html.Div(
                    dcc.Graph(figure={}, id='graph-id'),
#                     id='graph-id',
                     className='col s12 m6',
                ),
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                )
            ]
    )
])


#############################################
# Interaction Between Components / Controller
#############################################

# This callback will highlight a row on the data table when the user selects it
@app.callback(
#    [Output('datatable-id-container', 'children'),
    Output("datatable-id", "style_data_conditional"),
    Input("datatable-id", "derived_viewport_row_ids"),
    Input("datatable-id", 'selected_row_ids'),
    Input("datatable-id", 'active_cell'),
    Input("datatable-id", "derived_viewport_selected_rows"),
)
def selected(row_ids, selected_row_ids, active_cell, selected_rows):
    selected_id_set = set(selected_row_ids or [])
    print(selected_rows)

    if selected_rows is None:
        return []

    return (
            {
            'if': {'row_index': selected_rows},
            'backgroundColor': 'rgba(255,255,0,0.4)',
            'fontWeight': 'bold',
            #'color': 'white',
        },
    )


#This callback will highlight a cell on the data table when the user selects it
#This callback is not needed
# @app.callback(
#     Output('datatable-id', 'style_data_conditional'),
#     [Input('datatable-id', 'selected_columns')]
# )
# def update_styles(selected_columns):
#     return [{
#         'if': { 'column_id': i },
#         'background_color': '#D2F3FF'
#     } for i in selected_columns]


# This callback is used to update the dashboard based on radio button selection
# Note we have to use regex to catch abbreviated breed names in the .csv file
# For example, Chesapeake is abbreviated as "Chesa" in the .csv file
# Likewise many breed names have the work 'mix', so we need to catch the 
#    exact breed name match, as well as variations
@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):

    # FIX ME Add code to filter interactive data table with MongoDB queries
    # Only process dogs that meet the 'water rescue' requirements
    if filter_type == 'water':
        df = pd.DataFrame.from_records(db.readAll({
            "animal_type":"Dog",
            "breed": {
                '$in': [re.compile('.*Labrador.*'), re.compile('.*Chesa.*'), re.compile('.*Newfound.*')]
                # '$in': ["Labrador Retriever Mix", "Chesa Bay Retr Mix", "Newfoundland Mix"]
            },
            "sex_upon_outcome":'Intact Female',
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0},
        }))
        
    #only process dogs that meet the 'mountain/wilderness' requirements
    elif filter_type == 'wild':
        df = pd.DataFrame.from_records(db.readAll({
            "animal_type":"Dog",
            "breed": {
                '$in': [re.compile('.*German Shepherd.*'), re.compile('.*Alaskan Malamute.*'),
                        re.compile('.*Old English Sheepdog.*'), re.compile('.*Siberian Husky.*'),
                        re.compile('.*Rottweiler.*')]
                #'$in': ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                #        "Siberian Husky", "Rottweiler"]
            },
            "sex_upon_outcome":'Intact Male',
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0},
        }))
     
    #only process dogs that meet the 'disaster' requirements
    elif filter_type == 'disaster':
                df = pd.DataFrame.from_records(db.readAll({
            "animal_type":"Dog",
            "breed": {
                '$in':[re.compile('.*Doberman.*'), re.compile('.*German Shepherd.*'),
                       re.compile('.*Golden Retriever.*'), re.compile('.*Bloodhound.*'),
                       re.compile('.*Rottweiler.*')]
                # '$in': ["Doberman Pinsch", "German Shepherd", "Golden Retriever",
                #         "Bloodhound", "Rottweiler"]
            },
            "sex_upon_outcome":'Intact Male',
            "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0},
        }))

    # process all dogs
    elif filter_type == 'dogs':
                df = pd.DataFrame.from_records(db.readAll({
            "animal_type":"Dog",
            "sex_upon_outcome":'Intact Male',
            "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0},
        }))

    # default selection: process entire dataset
    else:
        df = pd.DataFrame.from_records(db.readAll({}))

    # ensure the 'id' column is dropped before returning
    df.drop(columns=['_id'],inplace=True)

    # prep columns to be returned
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]

    # set the results to be returned
    data=df.to_dict('records')

    # return the result set and columns
    return (data,columns)



# The following call back is used for the pie graph
# I used pie graph from plotly express (px.pie)
# Note that I used specialized styling including the following:
#  - specialized coloring based on names of outcome_types (9 different colors)
#  - placed text info inside each sector in radial position
#  - set vector text to uniform size, and "hide" text that won't fit in a sector
#  - stylized legend as follows:
#    -- set background color, border color, and border width
# Decision to base pie chart on outcome_types is to highlight dogs that are available
# (i.e., Grazioso is not interested in dogs that are deceased, euthanized, etc.)
@app.callback(
    Output('graph-id', 'figure'),
#    [Input('datatable-id', 'derived_viewport_data')])  # use this for current page
    [Input('datatable-id', 'derived_virtual_data')]) # use this for entire set
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    # Prevent and update if their is no data, otherwise capture the data into dff
    if viewData is None:
        raise PreventUpdate
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # set up names and values arrays that will be used by the pie chart
    nms = dff['outcome_type'].value_counts().keys().tolist()
    vals = dff['outcome_type'].value_counts().tolist()

    # configure the pie chart with my own color choices
    fig = px.pie(data_frame=dff, values=vals, names=nms, title='Outcome Status',
                 color=nms,
                     color_discrete_map={
                         'Adoption':'yellow',
                         'Rto-Adoption':'aquamarine',
                         'Transfer':'green',
                         'Return to Owner':'pink',
                         'Died':'red',
                         'Disposal':'crimson',
                         'Euthanasia':'silver',
                         'Missing':'brown',
                         'Relocate':'darkgoldenrod',
                     }
                 )
    
    # set up options for text positioning and orientation in the pie chart
    fig.update_traces(
        textposition='inside',
        textinfo='percent',
        insidetextorientation='radial'
    )
    
    # set up options for the layout of the pie chart
    fig.update_layout(
        uniformtext_minsize=12, uniformtext_mode='hide',
        legend=dict(
            #traceorder='reversed',
            #bgcolor='LightSteelBlue',
            bgcolor='navajowhite',
            bordercolor='black',
            borderwidth=2
        )
    )

    # past the chart (figure) back to caller
    return fig


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable

# To set up multiple markers I use a layergroup. 
# First create an array of markers from each row in current dataframe
# Each marker will get the lat/long, as well as tooltip and popup data from each row
# Then, the entire array of markers is passed to the leaflet layergroup function in the map
@app.callback (
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data'),
     Input('datatable-id', 'derived_viewport_selected_rows')]
)
def update_map(viewData, row_ids):
    
    # prevent updates if there is no data
    if viewData is None:
        print("viewData is none")
        raise PreventUpdate
    elif row_ids is None:
        print("index is none")
        raise PreventUpdate
    
    if (len(row_ids) == 0):
        raise PreventUpdate

    # if there is data in the viewport, then set it to dff
    dff = pd.DataFrame.from_dict(viewData)
    row_ids[0] = row_ids[len(row_ids) -1]
    
    # use a marker group to capture all of the locations in the viewport
    markers = [dl.Marker(position=[row["location_lat"], row["location_long"]],
                         children=[dl.Tooltip(row["breed"]),
                                   dl.Popup([html.H4("Name:"), html.H4(row["name"])])
                        ])
               for i, row in dff.iterrows()]
    
    # when returning the map, i am setting size, the center location
    # and passing the markers to the layergroup so that all five locations are set
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'}, 
               center=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], zoom=10, 
               children=[
                    dl.TileLayer(id="base-layer-id"),
                   dl.LayerGroup(markers),
                ], 
        )
    ]


# run the program
app.run_server(debug=True)


Chris Trimmer, CS340, Module 7 Project 2
aacuser and aac1234 from lib file
-> 10000 of 10000 document(s) read.


-> 10000 of 10000 document(s) read.
viewData is none
None
[0]
-> 10000 of 10000 document(s) read.
viewData is none
None
[0]
[0]
[2]
[4]
-> 19 of 10000 document(s) read.
-> 32 of 10000 document(s) read.
